In [ ]:
pip install face_recognition scikit-learn numpy pandas matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=d3477eb47e48de608d3dc088ed34140d71c4396157f256eeb4d5acf536f65f3f
  Stored in directory: /root/.cache/pip/wheels/8f/47/c8/f44c5aebb7507f7c8a2c0bd23151d732d0f0bd6884ad4ac635
Successfully built face-recognition-models


In [ ]:
import random
from sklearn.datasets import fetch_lfw_people

lfw = fetch_lfw_people(min_faces_per_person=20, resize=1.0)
X = lfw.images
y = lfw.target
names = lfw.target_names
selected_ids = list(range(10))
IMAGES_PER_ID = 20

images = []
labels = []

for i in selected_ids:
    idxs = [j for j in range(len(y)) if y[j] == i]
    idxs = random.sample(idxs, IMAGES_PER_ID)
    for j in idxs:
        images.append(X[j])
        labels.append(i)

In [ ]:
import face_recognition
import numpy as np
embeddings = []
valid_labels = []

for img, label in zip(images, labels):
    img_uint8 = (img * 255).astype(np.uint8)
    img_rgb = np.stack([img_uint8]*3, axis=-1)
    enc = face_recognition.face_encodings(img_rgb)
    if len(enc) == 1:
        embeddings.append(enc[0])
        valid_labels.append(label)

X_embed = np.array(embeddings)
y_embed = np.array(valid_labels)

print("Final samples:", X_embed.shape)

Final samples: (188, 128)


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train, X_test, y_train, y_test = train_test_split(X_embed, y_embed, test_size=0.3, stratify=y_embed, random_state=42)

print("Train size:", X_train.shape)
print("Test size:", X_test.shape)

clf = LogisticRegression(max_iter=2000)
clf.fit(X_train, y_train)

Train size: (131, 128)
Test size: (57, 128)


LogisticRegression(max_iter=2000)

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = clf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
acc = accuracy_score(y_test, y_pred)
print("Accuracy:", round(acc * 100, 2), "%")
print("Confusion Matrix:\n", cm)

TPR = []
FPR = []

for i in range(len(cm)):
    TP = cm[i, i]
    FN = cm[i, :].sum() - TP
    FP = cm[:, i].sum() - TP
    TN = cm.sum() - (TP + FN + FP)
    TPR_i = TP / (TP + FN) if (TP + FN) > 0 else 0
    FPR_i = FP / (FP + TN) if (FP + TN) > 0 else 0
    TPR.append(TPR_i)
    FPR.append(FPR_i)

print("Per-class TPR:", np.round(TPR, 3))
print("Per-class FPR:", np.round(FPR, 3))
print("Macro TPR:", round(np.mean(TPR), 4))
print("Macro FPR:", round(np.mean(FPR), 4))

Accuracy: 100.0 %
Confusion Matrix:
 [[6 0 0 0 0 0 0 0 0 0]
 [0 6 0 0 0 0 0 0 0 0]
 [0 0 6 0 0 0 0 0 0 0]
 [0 0 0 6 0 0 0 0 0 0]
 [0 0 0 0 6 0 0 0 0 0]
 [0 0 0 0 0 6 0 0 0 0]
 [0 0 0 0 0 0 5 0 0 0]
 [0 0 0 0 0 0 0 6 0 0]
 [0 0 0 0 0 0 0 0 5 0]
 [0 0 0 0 0 0 0 0 0 5]]
Per-class TPR: [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Per-class FPR: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Macro TPR: 1.0
Macro FPR: 0.0
